In [1]:
# скачиваем файл
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1VPkoWfiIvZl4HGp49BUaVEEblVIGYh91' -O heart.csv

--2021-12-27 14:47:45--  https://docs.google.com/uc?export=download&id=1VPkoWfiIvZl4HGp49BUaVEEblVIGYh91
Resolving docs.google.com (docs.google.com)... 172.217.203.139, 172.217.203.113, 172.217.203.138, ...
Connecting to docs.google.com (docs.google.com)|172.217.203.139|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0k-5k-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/frrtn6safcng8nrpsgud2qgstgnqk0us/1640616450000/07828702499945486090/*/1VPkoWfiIvZl4HGp49BUaVEEblVIGYh91?e=download [following]
--2021-12-27 14:47:45--  https://doc-0k-5k-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/frrtn6safcng8nrpsgud2qgstgnqk0us/1640616450000/07828702499945486090/*/1VPkoWfiIvZl4HGp49BUaVEEblVIGYh91?e=download
Resolving doc-0k-5k-docs.googleusercontent.com (doc-0k-5k-docs.googleusercontent.com)... 64.233.170.132, 2607:f8b0:400c:c0d::84
Connecting to doc-0k-5k-docs.googleusercontent.com (doc-0k-

In [2]:
# устанавливаем нужную версию библиотеки sklearn, чтобы ответы сошлись
!pip install scikit-learn==0.22.1

     |████████████████████████████████| 7.0 MB 4.8 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.1
    Uninstalling scikit-learn-1.0.1:
      Successfully uninstalled scikit-learn-1.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imbalanced-learn 0.8.1 requires scikit-learn>=0.24, but you have scikit-learn 0.22.1 which is incompatible.


In [3]:
import numpy as np
import pandas as pd

В этом задании вы будете работать с [данными о пациентах, у части которых есть заболевание сердца](https://www.kaggle.com/ronitf/heart-disease-uci). Вам нужно построить модель, классифицирующую пациентов на больных этим заболеванием и тех, у кого его нет, а также проанализировать результаты.

In [4]:
data = pd.read_csv('heart.csv')
data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


1. Какой процент пациентов из представленных в данных имеет заболевание сердца (`'target' = 1`)? Ответ округлите до двух знаков после запятой.

In [11]:
a = data['target'].to_list()
c = len(a)
c1 = a.count(1)
round(c1*100/c,2)

54.46

Разделим данную выборку на обучающую и тестовую части в отношении 3:1.

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.drop('target', axis=1), data['target'], test_size=0.25, random_state=13)
X_train.shX_testape, X_test.shape

((227, 13), (76, 13))

2. Обучите линейный классификатор из `sklearn` (`sklearn.linear_model.SGDClassifier`) с максимальным числом итераций `max_iter=1000`, постоянным значением шага градиентного спуска (`learning_rate='constant'`), равным `eta0=0.1`. В качестве сида поставьте `random_state=13`. Параметр регуляризации отключите: `alpha=0`. Класс `sklearn.linear_model.SGDClassifier` объединяет в себе разные линейные модели - чтобы получить логистическую регрессию, зафиксируйте параметр `loss='log'`.

   Подробнее о параметрах можно почитать в [документации](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html).

   Какое значение свободного коэффициента у полученного линейного классификатора? Ответ округлите до двух знаков после запятой.

In [17]:
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier(max_iter=1000, learning_rate='constant', eta0=0.1, random_state=13, alpha=0, loss='log')
clf.fit(X_train, y_train)
y_pred_sgdclf = clf.predict(X_test)

round(clf.intercept_[0],2)

-1.16

3. Какое значение L2-нормы вектора весов (без учета свободного коэффициента) у полученного линейного классификатора? Ответ округлите до двух знаков после запятой.

_Напоминание. L2-норма вектора $v = (v_1, \ldots, v_n)$ - это корень из суммы квадратов его элементов:_

$$
\|v\|_2 = \sqrt{\sum\limits_{i=1}^nv_i^2}
$$

In [35]:
c=0
for i in range(len(clf.coef_[0])):
  c += (clf.coef_[0,i])**2
round(c**0.5, 2)

175.68

4. Найдите долю правильных ответов классификатора на тестовой части выборки **(в процентах)**. Ответ округлите до двух знаков после запятой. Например, если значение доли правильных ответов будет равно 0.1234, то ответом будет 12.34 - ведь это 12.34%.

In [36]:
from sklearn.metrics import accuracy_score

round(accuracy_score(y_test,y_pred_sgdclf)*100,2)

61.84

5. В задаче классификации, как и в задаче регрессии, для оптимизации линейных моделей можно применять регуляризацию. Этот метод реализован и в `sklearn.linear_model.SGDClassifier` - параметр регуляризации обозначается параметром `alpha`. За тип регуляризации (L1, L2 или обе сразу) отвечает параметр `penalty`. 

   Обучите классификатор заново с параметром регуляризации `alpha=0.1` и типом регуляризации `penalty='l1'`. Оставьте максимальное число итераций, равное `max_iter=1000` и сид `random_state=13`. Также вместо постоянного значения шага градиентного спуска используйте оптимальное (`learning_rate='optimal'`), которое, кстати, зависит от `alpha` (о том, как именно он вычисляется и какие еще параметры можно выбрать, можно подробнее прочитать в [документации](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html)). В данном случае значение начального шага градиентного спуска `eta0` никак не участвует в обучении.
   
   Отличается ли качество полученного классификатора от качества первого? Какая доля правильных ответов получается теперь на тестовой выборке? Выразите ее **в процентах**, ответ округлите до двух знаков после запятой. Например, если значение доли правильных ответов будет равно 0.1234, то ответом будет 12.34 - ведь это 12.34%.

In [74]:
from sklearn.metrics import accuracy_score

clf2 = SGDClassifier(max_iter=1000, learning_rate='optimal', random_state=13, alpha=0.1, penalty='l1', loss='log')
clf2.fit(X_train, y_train)
y_pred = clf2.predict(X_test)

round(accuracy_score(y_test,y_pred)*100,2)

76.32

6. Найдите L2-норму вектора весов для полученного классификатора (заметьте, как на нее повлияла регуляризация). Ответ округлите до двух знаков после запятой.

    Заметьте, что вектор стал более разреженным, и в нем появились нулевые элементы - это результат действия L1-регуляризации.

In [75]:
c=0
for i in range(len(clf2.coef_[0])):
  c += (clf.coef_[0,i])**2
round(c**0.5, 2)

175.68

7. Наконец, проверьте, как полученные классификаторы предсказывают не классы, а вероятности классов - так как мы работаем с логистической регрессией, это можно сделать. Посмотрите на вероятности, которые выдает первый классификатор (обученный с постоянным шагом градиентного спуска и без регуляризации) на тестовой части выборки. В этом вам поможет метод `predict_proba`. Результатом его работы будет список размера $N\times 2$, где $N$ - это число объектов. В каждом столбце списка находятся вероятности соответствующего класса для объектов. Поэтому если вам нужен положительный класс, вас интересует последний столбец.

    Какое получается значение AUC-ROC? Ответ округлите до двух знаков после запятой.

In [76]:
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import roc_auc_score

clf.fit(X_train, y_train)
y_pred_sgdclf = clf.predict_proba(X_test)

round(roc_auc_score(y_test,y_pred_sgdclf[:, 1]),2)

0.57

8. Посмотрите на вероятности, которые выдает второй классификатор (обученный с оптимальным шагом градиентного спуска и с регуляризацией) на тестовой части выборки. Что вы наблюдаете - как отличаются эти вероятности от вероятностей первого классификатора?

   Посчитайте значение AUC-ROC второго классификатора. Ответ округлите до двух знаков после запятой.

In [77]:
clf2.fit(X_train, y_train)
y_pred_sgdclf = clf2.predict_proba(X_test)

round(roc_auc_score(y_test,y_pred_sgdclf[:, 1]),2)

0.86

9. Какой признак является самым важным по мнению лучшей модели (имеет наибольший по модулю коэффициент) для принятия решения?

In [89]:
clf2.coef_

array([[-0.87424016,  0.        ,  9.26085004, -0.45773776, -0.01501012,
         0.        ,  0.        ,  0.74467772,  0.        , -0.25903021,
         0.        , -4.1192229 , -2.1071261 ]])

In [92]:
a = max(clf2.coef_[0], key=lambda i: abs(i))

In [93]:
l = list(clf2.coef_[0])

In [94]:
l.index(a)

2

In [95]:
list(data)[l.index(a)]

'cp'